# 💡 07. 원-핫 인코딩 (One-hot Encoding): 스위치 켜기

## 1. 개요
앞서 단어를 숫자(1, 2, 3...)로 바꿨습니다. 
그런데 숫자로 그냥 두면 AI가 **"3번 단어는 1번 단어보다 3배 더 크네?"**라고 오해할 수 있습니다.
이 오해를 막기 위해, 숫자의 크기를 없애고 **위치**로만 표시하는 방법이 바로 **원-핫 인코딩**입니다.

> **쉬운 비유**:  
> - **정수 인코딩**: 1번, 2번, 3번 선수 (등수가 아님에도 등수처럼 보임)
> - **원-핫 인코딩**: 전구 100개 중 **해당하는 전구 1개만 스위치를 켜는(ON)** 방식.
>   - 사과 ➡️ `[1, 0, 0]`
>   - 바나나 ➡️ `[0, 1, 0]`
>   - 딸기 ➡️ `[0, 0, 1]`

---

## 2. 주요 개념 및 장단점

### 특징
- 단어 집합의 크기만큼 길쭉한 벡터를 만듭니다.
- 딱 1개의 값만 `1`(Hot)이고, 나머지는 모두 `0`(Cold)입니다.

### 💥 치명적인 단점 (메모리 폭발)
- 단어가 10,000개 있으면, 벡터 길이도 10,000칸이 됩니다.
- `[0, 0, ..., 1, ..., 0]` : 0이 너무 많아서 저장 공간 낭비가 심합니다. (**희소 행렬**이라고 합니다)
- 단어 간의 관련성을 알 수 없습니다. (사과와 배가 비슷한지 모름)

> 그래서 요즘은 **임베딩(Embedding)**이라는 더 좋은 방법을 주로 쓰지만, 기본 원리를 이해하기 위해 배워봅시다.

## 3. 실습해보기

먼저 한국어 형태소 분석기로 문장을 쪼개고 정수 인코딩을 만듭니다.

In [ ]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = "오늘 점심은 맛있는 김밥이다"

# 1. 토큰화
okt = Okt()
tokens = okt.morphs(text)
print("토큰:", tokens)

# 2. 정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts([tokens]) # 리스트로 감싸서 넣어야 함
print("단어장:", tokenizer.word_index)

sequences = tokenizer.texts_to_sequences([tokens])[0]
print("정수 인코딩:", sequences)

## 4. `to_categorical`로 변환하기
Keras의 `to_categorical` 함수를 쓰면 정수를 원-핫 벡터로 바꿔줍니다.

In [ ]:
# 문장의 길이는 4개, 단어 종류는 4개입니다.
# to_categorical(정수 데이터, 전체 클래스 개수)
# 주의: 인덱스가 1부터 시작하므로, 클래스 개수는 (가장 큰 번호 + 1)이어야 안전합니다.

vocab_size = len(tokenizer.word_index) + 1
one_hot = to_categorical(sequences, num_classes=vocab_size)

print("결과 행렬 크기:", one_hot.shape)
print("\n원-핫 벡터들:")
print(one_hot)

### 결과 해석
- `[0. 1. 0. 0. 0.]` ➡️ 1번 단어 ('오늘')
- `[0. 0. 1. 0. 0.]` ➡️ 2번 단어 ('점심')
- 맨 앞의 `0.`은 인덱스 0번(패딩용) 자리를 의미합니다.

## 5. 원-핫 인코딩의 한계 직접 확인하기
단어 사이의 유사도를 계산(내적)해보면 무조건 0이 나옵니다. 서로 완전히 남남이라는 뜻이죠.

In [ ]:
import numpy as np

vec1 = one_hot[0] # 오늘
vec2 = one_hot[1] # 점심

# 내적(dot product) 계산
similarity = np.dot(vec1, vec2)
print(f"'오늘'과 '점심'의 유사도: {similarity}")

# 결과가 0이라는 것은 '전혀 관계없음'을 뜻합니다.
# 실제로는 관계가 있지만, 원-핫 인코딩은 이걸 표현 못 합니다.

## 마무리

- **One-hot Encoding**: 스위치를 하나만 켜는 방식.
- **단점**: 공간 낭비 심함, 단어의 속뜻을 모름.
- **해결책**: 다음에 배울 **워드 임베딩(Word Embedding)**!